In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)

#from pandas import DataFrame, read_csv
import pandas as pd

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics

import sys  
#sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
from AdaBandFlt import *

%matplotlib tk

In [3]:
#load data
# file path of csv file
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'
Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [4]:
#load data
# file path of csv file
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2_tstamp.csv'
Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2_tstamp.csv'

# create dataframe
df_ = pd.read_csv(Location, sep=',')

In [5]:
df.head()

,El 21,El 31,El 41,El 22,El 32,El 42,El 23,El 33,El 43,El 15
%t,,,,,,,,,,
0.00,-2.61,1.02,0.80,1.93,-1.25,-1.59,-0.57,4.09,-1.14,-0.34
0.04,-3.30,0.00,2.05,1.25,-1.02,-0.45,-1.82,2.61,-0.45,-0.80
0.08,-2.73,0.11,2.73,1.48,0.45,1.14,-1.14,3.52,-0.23,-0.80
0.12,-3.07,-0.45,1.25,0.80,-0.57,-0.57,-1.25,2.50,-1.02,-0.57
0.16,-2.39,1.48,0.34,1.14,0.00,-0.57,-0.34,1.48,-0.23,0.45


In [6]:
df_.head()

,T[s],sppa0001_21_Timestamp_Unit0,T[s].1,sppa0001_31_Timestamp_Unit0,T[s].2,sppa0001_41_Timestamp_Unit0,T[s].3,sppa0001_22_Timestamp_Unit0,T[s].4,sppa0001_32_Timestamp_Unit0,T[s].5,sppa0001_42_Timestamp_Unit0,T[s].6,sppa0001_23_Timestamp_Unit0,T[s].7,sppa0001_33_Timestamp_Unit0,T[s].8,sppa0001_43_Timestamp_Unit0,T[s].9,sppa0001_15_Timestamp_Unit0
0,1.63776,NaN,0.38532,NaN,0.85184,NaN,2.14396,NaN,2.13620,NaN,0.56064,NaN,2.14936,NaN,0.47808,NaN,2.13872,NaN,7.96416,NaN
1,1.99384,NaN,1.08808,NaN,2.12424,NaN,2.15492,NaN,2.14376,NaN,2.14380,NaN,2.15504,NaN,0.90124,NaN,2.14168,NaN,16.50048,NaN
2,2.13476,NaN,2.11608,NaN,2.13104,NaN,2.16148,NaN,2.15196,NaN,2.15204,NaN,2.16224,NaN,2.11860,NaN,2.14432,NaN,17.73908,NaN
3,2.13824,NaN,2.12060,NaN,2.13352,NaN,2.16732,NaN,2.15752,NaN,2.15772,NaN,2.17080,NaN,2.14616,NaN,2.15276,NaN,NaN,NaN
4,2.14420,NaN,2.13140,NaN,2.13628,NaN,2.17520,NaN,2.16072,NaN,2.16072,NaN,2.17296,NaN,2.14848,NaN,2.15540,NaN,NaN,NaN


In [7]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

#xminnoise = int(np.round(11114*(fs/1000)))
#xmaxnoise = int(np.round(18511*(fs/1000)))

#noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

#xminspike = int(np.round(130826*(fs/1000)))
#xmaxspike = int(np.round(131699*(fs/1000)))

#burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

#begin_data = signal_filtered.iloc[:500000]

C:\Users\SYL21\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [8]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered

In [9]:
fs = 25000

noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

"""plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')"""

"plt.figure()\nplt.plot(noise_levels)\nplt.grid(True)\nplt.xlabel('Time')\nplt.ylabel('Noise Amplitude [µV]')\nplt.title('Noise Levels')"

In [10]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.0012, 
                           noise_window_size = 0.01,
                           threshold_factor = 3.5,
                           maxseparation = 0.001)  
                
#spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']
spike_info

,indice_max,indice_min,indice_depass_positif,indice_depass_negatif,indice_1er_depass,indice_zero_central,i_max-i_min,Delta_amplitudes
0,53278,53286,53277,53285,53277,53281,-8,16.237153
1,53391,53375,53389,53373,53373,53386,16,18.401586
2,53492,53504,53489,53499,53489,53497,-12,31.926962
3,53645,53652,53643,53652,53643,53649,-7,17.140363
4,53922,53939,53910,53931,53910,53927,-17,44.081780
5,53960,53952,53959,53948,53948,53957,8,28.400890
6,54115,54133,54106,54126,54106,54123,-18,61.367703
7,54261,54248,54256,54245,54245,54253,13,27.291692
8,54314,54329,54302,54322,54302,54320,-15,59.721773
9,54481,54491,54464,54488,54464,54486,-10,49.451979


In [11]:
signal = signal_filtered

In [53]:
toto = df_.iloc[:, 2].dropna().values
arr = toto*25000
spike_info_ = pd.DataFrame(arr.astype('int') , columns = ['indice_zero_central'])


In [27]:
signal = full_signal

In [55]:
spike_data = record_spikes(signal, fs, spike_info,
                  'indice_zero_central',
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_ = record_spikes(signal, fs, spike_info_,
                  'indice_zero_central',
                  t_before = 0.001,
                  t_after = 0.002)

(377, 76)
(466, 76)


In [39]:
def print_spikes(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = 25000,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.index-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()

In [64]:
sophie_spike = spike_info_['indice_zero_central'].values
our_spike = spike_info['indice_zero_central'].values
temporal_tol = 0.0006

i = 0
j = 0

joint_spikes_sophie_side = []
solo_spikes_sophie_side = []
joint_spikes_our_side = []
solo_spikes_our_side = []
tol = int(np.round(fs * temporal_tol))

while(True):
    if sophie_spike[i] < our_spike[j] - tol:
        solo_spikes_sophie_side.append(i)
        i += 1
    elif sophie_spike[i] > our_spike[j] + tol:
        solo_spikes_our_side.append(j)
        j += 1
    else:
        joint_spikes_sophie_side.append(i)
        joint_spikes_our_side.append(j)
        i += 1
        j += 1
    if(i == len(sophie_spike)-1 or j == len(our_spike)-1):
        break
    
print(len(sophie_spike))
print(len(our_spike))
print('joint_spikes_sophie_side')
print(len(joint_spikes_sophie_side))
print(joint_spikes_sophie_side)
print('solo_spikes_sophie_side')
print(len(solo_spikes_sophie_side))
print(solo_spikes_sophie_side)
print('joint_spikes_our_side')
print(len(joint_spikes_our_side))
print(joint_spikes_our_side)
print('solo_spikes_our_side')
print(len(solo_spikes_our_side))
print(solo_spikes_our_side)

465
376
joint_spikes_sophie_side
182
[4, 20, 22, 23, 25, 27, 29, 35, 37, 38, 39, 43, 44, 45, 49, 51, 55, 58, 59, 61, 65, 69, 72, 73, 76, 77, 78, 79, 80, 84, 89, 91, 92, 95, 96, 97, 98, 99, 102, 103, 104, 107, 108, 110, 113, 115, 117, 118, 122, 123, 124, 125, 126, 127, 128, 129, 133, 137, 138, 139, 140, 141, 146, 147, 153, 155, 159, 161, 165, 168, 170, 179, 180, 184, 185, 186, 190, 193, 196, 199, 202, 211, 212, 214, 215, 219, 224, 225, 228, 232, 235, 236, 241, 245, 246, 247, 250, 251, 252, 253, 255, 259, 261, 265, 268, 272, 280, 282, 283, 288, 291, 293, 301, 303, 307, 310, 311, 313, 314, 316, 317, 319, 321, 323, 325, 326, 329, 330, 331, 332, 336, 338, 341, 342, 343, 346, 348, 350, 353, 356, 357, 358, 360, 364, 368, 371, 372, 374, 375, 376, 380, 382, 396, 399, 400, 401, 403, 404, 405, 406, 407, 410, 412, 415, 416, 418, 422, 423, 427, 429, 430, 431, 436, 437, 438, 439, 442, 446, 447, 452, 456, 459]
solo_spikes_sophie_side
278
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,

In [66]:
print_spikes(spike_data_)

In [60]:
spike_data_oneline_joint_our_side = record_spikes_oneline(signal, fs, spike_info.iloc[joint_spikes_our_side],
                  'indice_zero_central',
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_oneline_solo_our_side = record_spikes_oneline(signal, fs, spike_info.iloc[solo_spikes_our_side],
                  'indice_zero_central',
                  t_before = 0.001,
                  t_after = 0.002)

In [61]:
figure = plt.figure()
axes = figure.add_subplot(2, 1, 1)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_joint_our_side.index, spike_data_oneline_joint_our_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_joint_our_side')
axes.grid(True)
axes = figure.add_subplot(2, 1, 2)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_solo_our_side.index, spike_data_oneline_solo_our_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_solo_our_side')
axes.grid(True)

In [62]:
spike_data_oneline_joint_sophie_side = record_spikes_oneline(signal, fs, spike_info_.iloc[joint_spikes_sophie_side],
                  'indice_zero_central',
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_oneline_solo_sophie_side = record_spikes_oneline(signal, fs, spike_info_.iloc[solo_spikes_sophie_side],
                  'indice_zero_central',
                  t_before = 0.001,
                  t_after = 0.002)

In [63]:
figure = plt.figure()
axes = figure.add_subplot(2, 1, 1)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_joint_sophie_side.index, spike_data_oneline_joint_sophie_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_joint_sophie_side')
axes.grid(True)
axes = figure.add_subplot(2, 1, 2)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_solo_sophie_side.index, spike_data_oneline_solo_sophie_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_solo_sophie_side')
axes.grid(True)